# Recipe Recommender Pipeline Demonstration  
### YOLO → Ingredient Normalization → Recipe Matching

This notebook demonstrates how to use the **RecipeRecommender** pipeline, which fully integrates:

- **YOLOv8 ingredient detection**
- **Canonical ingredient normalization**
- **Normalized recipe database (pre-computed PKL)**
- **Simple and predictable recipe matching**
- **End-to-end pipeline for backend consumption**

This notebook is intended as a handoff for the teammate implementing:

- Backend API (FastAPI/Flask)
- UI (Streamlit, HTML/JS, Mobile, etc.)
- Docker deployment
- Basically, getting the app to have an UI and able to spin up & call the recipe recommender located below

## Below is a demonstration of RecipeRecommender (src/backend)

### Step 1: Import Recommender

In [5]:
from src.backend.recipe_recommender import RecipeRecommender

# Initialize the unified pipeline
rr = RecipeRecommender()

print("RecipeRecommender initialized with", len(rr.recipe_dict), "recipes.")


[RecipeRecommender] Loaded 13244 normalized recipes.
RecipeRecommender initialized with 13244 recipes.


### Step 2: Run Single Image Recommendation

In [6]:
import os
import random

# Pick a random fridge image from the dataset
test_folder = "data/fridge_photos/test/images"
test_image = os.path.join(test_folder, random.choice(os.listdir(test_folder)))

print("Using test image:", test_image)

result = rr.recommend(test_image, top_k=10)
result


Using test image: data/fridge_photos/test/images/DSC_6120_JPG_jpg.rf.27ad8951633df4437153aba393de06d6.jpg

=== Pipeline Start ===
Image: data/fridge_photos/test/images/DSC_6120_JPG_jpg.rf.27ad8951633df4437153aba393de06d6.jpg

image 1/1 /home/platinumfish/Desktop/6700/project/ml-app-deployment-proj/data/fridge_photos/test/images/DSC_6120_JPG_jpg.rf.27ad8951633df4437153aba393de06d6.jpg: 640x640 1 chicken, 1 chicken_breast, 1 eggs, 1 flour, 1 green_beans, 1 ground_beef, 1 milk, 1 mushrooms, 1 onion, 2 potatos, 1 spinach, 1 strawberries, 1 tomato, 294.3ms
Speed: 6.9ms preprocess, 294.3ms inference, 4.4ms postprocess per image at shape (1, 3, 640, 640)

[1] Raw YOLO detections: ['chicken', 'flour', 'onion', 'eggs', 'green_beans', 'ground_beef', 'milk', 'spinach', 'potato', 'tomato', 'strawberries', 'chicken_breast', 'mushrooms']
[2] Canonical fridge items: ['chicken', 'chicken_breast', 'eggs', 'flour', 'green_beans', 'ground_beef', 'milk', 'mushrooms', 'onion', 'potato', 'spinach', 'strawbe

{'image_path': 'data/fridge_photos/test/images/DSC_6120_JPG_jpg.rf.27ad8951633df4437153aba393de06d6.jpg',
 'fridge_items': ['chicken',
  'chicken_breast',
  'eggs',
  'flour',
  'green_beans',
  'ground_beef',
  'milk',
  'mushrooms',
  'onion',
  'potato',
  'spinach',
  'strawberry',
  'tomato'],
 'recommendations': [{'title': 'Portobello Mushroom Wellington',
   'score': 5.0,
   'ingredients': ['butter',
    'eggs',
    'flour',
    'milk',
    'mushrooms',
    'rosemary',
    'spinach']},
  {'title': 'Old-Fashioned Scalloped Potatoes',
   'score': 4.0,
   'ingredients': ['butter', 'flour', 'milk', 'onion', 'potato']},
  {'title': 'Roasted Strawberry Layer Cake',
   'score': 4.0,
   'ingredients': ['butter',
    'eggs',
    'flour',
    'milk',
    'pasta',
    'strawberry',
    'sugar',
    'zest of lemon']},
  {'title': 'Muffin Cup Veggie Omelets',
   'score': 4.0,
   'ingredients': ['cheese',
    'eggs',
    'mushrooms',
    'onion',
    'pepper',
    'salt',
    'spinach']},
  {

### YOLO Detection Alone

In [7]:
# Raw YOLO detection only
raw_labels = rr.detector.detect(test_image)
raw_labels



image 1/1 /home/platinumfish/Desktop/6700/project/ml-app-deployment-proj/data/fridge_photos/test/images/DSC_6120_JPG_jpg.rf.27ad8951633df4437153aba393de06d6.jpg: 640x640 1 chicken, 1 chicken_breast, 1 eggs, 1 flour, 1 green_beans, 1 ground_beef, 1 milk, 1 mushrooms, 1 onion, 2 potatos, 1 spinach, 1 strawberries, 1 tomato, 545.7ms
Speed: 5.8ms preprocess, 545.7ms inference, 2.7ms postprocess per image at shape (1, 3, 640, 640)


['chicken',
 'flour',
 'onion',
 'eggs',
 'green_beans',
 'ground_beef',
 'milk',
 'spinach',
 'potato',
 'tomato',
 'strawberries',
 'chicken_breast',
 'mushrooms']

### Canonical Normalization Only

In [8]:
# Canonical ingredient normalization (fuzzy → canonical_vocab)
canonical = rr.preprocessor.normalize(raw_labels)
canonical

['chicken',
 'chicken_breast',
 'eggs',
 'flour',
 'green_beans',
 'ground_beef',
 'milk',
 'mushrooms',
 'onion',
 'potato',
 'spinach',
 'strawberry',
 'tomato']

### Show Recipe Ingredient Structure

In [44]:
# View one random recipe and its normalized ingredients (from PKL)
import random

rand_title = random.choice(list(rr.recipe_dict.keys()))
rand_ingredients = rr.recipe_dict[rand_title]

print("Title:", rand_title)
print("Normalized Ingredients:", rand_ingredients)


Title: Caramelized Chipotle Chicken
Normalized Ingredients: ['cinnamon', 'mustard', 'sugar']


### Simple Overlap Matching Example

In [10]:
# Demonstrating the simple 1-1 scoring mechanism
fridge_items = ["milk", "eggs", "flour"]

example_recipe = ["milk", "eggs", "sugar", "butter"]

score = rr.matcher.match(fridge_items, example_recipe)

print("Fridge items:      ", fridge_items)
print("Recipe ingredients:", example_recipe)
print("Match score:", score)


Fridge items:       ['milk', 'eggs', 'flour']
Recipe ingredients: ['milk', 'eggs', 'sugar', 'butter']
Match score: 2


### Batch Recommendation Example

In [45]:
# Running recommendation on multiple images (batch)
results_batch = []

for img in os.listdir(test_folder)[:3]:
    image_path = os.path.join(test_folder, img)
    out = rr.recommend(image_path, top_k=3)
    results_batch.append(out)

results_batch



=== Pipeline Start ===
Image: data/fridge_photos/test/images/DSC_6100_JPG_jpg.rf.75297e97f555554bacdb6caf8a06453b.jpg

image 1/1 /home/platinumfish/Desktop/6700/project/ml-app-deployment-proj/data/fridge_photos/test/images/DSC_6100_JPG_jpg.rf.75297e97f555554bacdb6caf8a06453b.jpg: 640x640 1 apple, 1 beef, 1 bread, 1 carrot, 1 chicken, 1 chicken_breast, 1 corn, 1 eggs, 1 flour, 1 ground_beef, 1 milk, 1 sugar, 523.7ms
Speed: 8.3ms preprocess, 523.7ms inference, 7.1ms postprocess per image at shape (1, 3, 640, 640)

[1] Raw YOLO detections: ['beef', 'sugar', 'milk', 'flour', 'corn', 'chicken', 'carrot', 'apple', 'bread', 'chicken_breast', 'eggs']
[2] Canonical fridge items: ['apple', 'beef', 'bread', 'carrot', 'chicken', 'chicken_breast', 'corn', 'eggs', 'flour', 'milk', 'sugar']

=== Top Matches ===
- Big Apple Crumble Cupcakes (score=5.0000)
  ingredients: ['apple', 'butter', 'cinnamon', 'confectioners sugar', 'eggs', 'flour', 'granulated sugar', 'milk', 'salt', 'sugar', 'water']
- Fres

[{'image_path': 'data/fridge_photos/test/images/DSC_6100_JPG_jpg.rf.75297e97f555554bacdb6caf8a06453b.jpg',
  'fridge_items': ['apple',
   'beef',
   'bread',
   'carrot',
   'chicken',
   'chicken_breast',
   'corn',
   'eggs',
   'flour',
   'milk',
   'sugar'],
  'recommendations': [{'title': 'Big Apple Crumble Cupcakes',
    'score': 5.0,
    'ingredients': ['apple',
     'butter',
     'cinnamon',
     'confectioners sugar',
     'eggs',
     'flour',
     'granulated sugar',
     'milk',
     'salt',
     'sugar',
     'water']},
   {'title': 'Fresh Corn Pancakes',
    'score': 5.0,
    'ingredients': ['corn', 'eggs', 'flour', 'milk', 'sugar']},
   {'title': 'Roasted Strawberry Layer Cake',
    'score': 4.0,
    'ingredients': ['butter',
     'eggs',
     'flour',
     'milk',
     'pasta',
     'strawberry',
     'sugar',
     'zest of lemon']}]},
 {'image_path': 'data/fridge_photos/test/images/DSC_6108_JPG_jpg.rf.a6f78ec4a2a292e834a662a018ba7f75.jpg',
  'fridge_items': ['apple',

### Format Top Recipes

In [ ]:
# Show formatted recipe results
for rec in result["recommendations"]:
    print(f"{rec['title']} (score={rec['score']})")
    print("  ingredients:", rec["ingredients"])

Portobello Mushroom Wellington (score=5.0)
  ingredients: ['butter', 'eggs', 'flour', 'milk', 'mushrooms', 'rosemary', 'spinach']
Old-Fashioned Scalloped Potatoes (score=4.0)
  ingredients: ['butter', 'flour', 'milk', 'onion', 'potato']
Roasted Strawberry Layer Cake (score=4.0)
  ingredients: ['butter', 'eggs', 'flour', 'milk', 'pasta', 'strawberry', 'sugar', 'zest of lemon']
Muffin Cup Veggie Omelets (score=4.0)
  ingredients: ['cheese', 'eggs', 'mushrooms', 'onion', 'pepper', 'salt', 'spinach']
Strawberry Shortcake Cupcakes (score=4.0)
  ingredients: ['butter', 'cornstarch', 'eggs', 'flour', 'milk', 'salt', 'strawberry', 'sugar']
Thanksgiving Skillet Pizza (score=4.0)
  ingredients: ['chicken', 'mushrooms', 'onion', 'pepper', 'potato']
Sweet Potato Bread with Caramel and Aleppo-Spiced Pecans (score=4.0)
  ingredients: ['eggs', 'flour', 'milk', 'pecans', 'potato', 'salt', 'sugar']
Chicken and Dumplings with Mushrooms (score=4.0)
  ingredients: ['bay leaves', 'eggs', 'flour', 'low sodi

# Integration Example (FastAPI / Backend)

Your teammate (Hi Samantha) can integrate the recommender like this:

```python
from fastapi import FastAPI, UploadFile
from src.backend.recipe_recommender import RecipeRecommender
import tempfile

app = FastAPI()
rr = RecipeRecommender()

@app.post("/recommend")
async def recommend(file: UploadFile):
    temp = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
    temp.write(await file.read())
    temp.close()

    result = rr.recommend(temp.name, top_k=5)
    return result

### This end point will return .json:

```json
{
  "image_path": "...jpg",
  "fridge_items": ["milk","eggs","flour"],
  "recommendations": [
    {
      "title": "Fresh Corn Pancakes",
      "score": 4,
      "ingredients": ["corn","milk","eggs","flour","sugar"]
    }
  ]
}
```

## **Final Team Handoff Instructions**


## ✅ What Allen Completed (ML Pipeline)
### 1. YOLOv8 Ingredient Detection  
- Fully integrated with ultralytics  
- Works on all test fridge images  
- Produces accurate ingredient labels

### 2. Canonical Ingredient Normalization  
- Converts YOLO labels → clean canonical tokens  
- Uses fuzzy matching + canonical_vocab.json

### 3. Recipe Normalization (60k+ ingredients cleaned)  
- All recipes have preprocessed ingredient lists  
- Cached into:
  - `data/normalized_recipes.pkl` (runtime)
  - `data/cached_normalized.csv` (debug)

### 4. Recipe Matching  
- Simple overlap scoring restored  
- Stable and predictable recommendations  
- Pipeline ready for backend integration

### 5. Unified Recommender Object  
- Complete end-to-end pipeline:
- YOLO → canonical → normalized recipes → matching → top results
- Lives in:  
  **src/backend/recipe_recommender.py**

---

# What Teammate Needs To Build

## 1. Build REST API Backend (FastAPI recommended)
Endpoints:
- `POST /detect` → returns detected ingredients
- `POST /recommend` → returns top recipes
- `GET /health` → healthcheck (required for Docker)

Backend must:
- Handle image uploads
- Save images temporarily
- Call RecipeRecommender
- Return JSON responses
- Validate inputs

---

## 2. Build Frontend / UI
Tools allowed:
- Streamlit
- React
- HTML/JS frontend
- Flask/Bootstrap
- FastAPI + Jinja templates

UI should:
- Upload an image
- Preview image + YOLO detections
- Show recommended recipes
- Show matching ingredients
- Show missing ingredients

Optional:
- Multi-image mode
- Ingredient “pantry list”
- Dietary filters (vegan, dairy-free, etc.)

---

## 3. Docker Deployment
Team must build Dockerfile including:
```python
FROM python:3.10
RUN pip install ultralytics fastapi uvicorn torch ...
COPY src /app/src
COPY data /app/data
CMD ["uvicorn", "backend.main:app", "--host", "0.0.0.0", "--port", "8000"]
```

Deliverables:
- Dockerfile
- docker-compose.yml
- README for running locally

---

## 4. Final Integration & Testing
✔ API endpoints tested with sample images  
✔ Frontend tested with full pipeline  
✔ Docker image builds locally  
✔ Optional: deploy to cloud (Render, AWS, Lightsail, etc.)

---

# Summary

This notebook + codebase provides a **complete ML backend**.

Your teammate now only needs to:

### **Build the WEB BACKEND + UI + Docker deploy**  
Everything else (model, recipe logic, detection pipeline) is fully ready.

---
